In [11]:
import joblib
EXP_PATH_NAME="SCNN-IF"
RIVAL_PATH_NAME="WACA-IF"
joblib.cpu_count()

32

In [12]:
# !pip install --upgrade pip

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import gc

from joblib import Parallel, delayed

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dataclasses
from sklearn.svm import OneClassSVM
from dataclasses import asdict
from tqdm import tqdm
import warnings
import random
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold # Feature selector

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns


# Global utitlity functions are in separate notebook
%run ./Classification_utility-functions.ipynb
%run ./SEED-CONSTANTS.ipynb

np.random.seed(SEED)
print(f"Numpy Seed was set to: {SEED}")

print("Setup Complete")

SEED: 567
FINAL_EXP_RESULTS_PATH: clip=False_experiments_results
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
EER: 0.333, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.400, Threshold: 0.200 <-- Worse case
EER: 0.167, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.333, Threshold: 1.000 <-- Worse case
--------------------Utility functions imported--------------------
SEED: 567
FINAL_EXP_RESULTS_PATH: clip=False_experiments_results
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Preprocessing utility functions imported--------------------
SEED: 567
FINAL_EXP_RESULTS_PATH: clip=False_experiments_results
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Neu

In [13]:
import sklearn
sklearn.__dir__()
sklearn.__version__

'1.0.2'

In [14]:
@dataclasses.dataclass
class ExperimentParameters:
    """Contains all relevant parameters to run an experiment."""

    name: str  # Name of Parameter set. Used as identifier for charts etc.
    frequency: int
    max_subjects: int
    max_test_subjects: int
        
    user_ids: list
    num_sample_points_per_exp: int
    exp_begin_cutoff_idx: int
    exp_end_cutoff_idx: int
        
    
    seconds_per_subject_train: float
    seconds_per_subject_test: float
    window_size: int  # After resampling
    ocsvm_step_width: int  # After resampling
    scaler: str  # StandardScaler, MinMaxScaler, Normalizer, MaxAbsScaler, RobustScaler, PowerTransformer
    scaler_scope: str  # {"subject", "session"}
    scaler_global: bool  # fit transform scale on all data (True) or fit on training only (False)
    ocsvm_kernel: str # ocsvm kernel
    ocsvm_nu: float  # Best value found in random search, used for final model
    ocsvm_gamma: float  # Best value found in random search, used for final model
    feature_cols: list  # Columns used as features
    exclude_subjects: list  # Don't load data from those users
        
    # Calculated values
    def __post_init__(self):
        # HDF key of table:
        self.table_name = f"sensors_{self.frequency}hz"

        

# INSTANCES
# ===========================================================

# NAIVE_APPROACH
# -----------------------------------------------------------
NAIVE_MINMAX_OCSVM = ExperimentParameters(
    name="NAIVE-MINMAX_OCSVM",
    frequency=100,
    max_subjects=29,
    max_test_subjects=10,
    user_ids = [1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 28, 29, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49],
    num_sample_points_per_exp=21000,
    exp_begin_cutoff_idx=500,
    exp_end_cutoff_idx=-500,
    seconds_per_subject_train=210,
    seconds_per_subject_test=210,    
    window_size=250,
    ocsvm_step_width=250,
    scaler="minmax",
    scaler_scope="subject",
    scaler_global=True,
    ocsvm_kernel="rbf",
    ocsvm_nu=None,
    ocsvm_gamma=None,
    feature_cols=[
        "x_a",
        "y_a",
        "z_a",
        "x_g",
        "y_g",
        "z_g",
    ],
    exclude_subjects=[],
)

# VALID_APPROACH
# -----------------------------------------------------------
VALID_MINMAX_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-MINMAX-OCSVM",
    scaler_global=False,
    ocsvm_nu=0.165,
    ocsvm_gamma=0.039,
)

# NAIVE_ROBUST_APPROACH
# -----------------------------------------------------------
NAIVE_ROBUST_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="NAIVE-ROBUST-OCSVM",
    scaler="robust",
    scaler_global=True,
    ocsvm_nu=0.153,
    ocsvm_gamma=0.091,  # below median, selected by chart
)

# ROBUST_APPROACH (VALID)
# -----------------------------------------------------------
VALID_ROBUST_OCSVM_125 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=125
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)


VALID_ROBUST_OCSVM_250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_2000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=2000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

# NORMALIZER_APPROACH (VALID)
# -----------------------------------------------------------
VALID_NORMALIZER_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-NORMALIZER-OCSVM",
    scaler="Normalizer",
    scaler_global=False,
    ocsvm_nu=0.074,
    ocsvm_gamma= 0.029,
)

In [15]:
P = VALID_ROBUST_OCSVM_2000
P.ocsvm_step_width = int(P.window_size * .5)
P.classifier="IF"

P.train_cores=1 # 20 cores for every user and 1 core for the nested crossval function
P.test_cores=2 # 10 cores for every user and 2 for the nested crossval function


# param_dist
param_dist = {
    "model__n_estimators": np.linspace(50, 200, num=10, dtype=np.int64),
    'model__max_samples':  np.arange(0.1, 1.1, 0.1),
    'model__contamination': np.arange(0.01, 0.11, 0.01),
    'model__max_features': np.arange(0.1, 1.1, 0.1),
    # 'model__bootstrap': [True, False], 
    }

param_dist = {
    "model__n_estimators": [10],
    'model__max_samples':  np.arange(0.1, 1.1, 0.1),
    'model__contamination': np.arange(0.01, 0.11, 0.01),
    'model__max_features': np.arange(0.1, 1.1, 0.1),
    # 'model__bootstrap': [True, False], 
    }

param_dist = {
    "model__n_estimators": [50, 100, 150, 200],
    'model__max_samples':  np.arange(0.1, 1.01, 0.1),
    'model__contamination': np.linspace(0.0001, 0.1, 50),
    'model__max_features': np.arange(0.1, 1.01, 0.1),
    'model__warm_start': [False], 
    }

P.scaler_clip=False

P.is_NN=True

In [16]:
param_dist

{'model__n_estimators': [50, 100, 150, 200],
 'model__max_samples': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 'model__contamination': array([0.0001    , 0.00213878, 0.00417755, 0.00621633, 0.0082551 ,
        0.01029388, 0.01233265, 0.01437143, 0.0164102 , 0.01844898,
        0.02048776, 0.02252653, 0.02456531, 0.02660408, 0.02864286,
        0.03068163, 0.03272041, 0.03475918, 0.03679796, 0.03883673,
        0.04087551, 0.04291429, 0.04495306, 0.04699184, 0.04903061,
        0.05106939, 0.05310816, 0.05514694, 0.05718571, 0.05922449,
        0.06126327, 0.06330204, 0.06534082, 0.06737959, 0.06941837,
        0.07145714, 0.07349592, 0.07553469, 0.07757347, 0.07961224,
        0.08165102, 0.0836898 , 0.08572857, 0.08776735, 0.08980612,
        0.0918449 , 0.09388367, 0.09592245, 0.09796122, 0.1       ]),
 'model__max_features': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 'model__warm_start': [False]}

In [17]:
utils_ppp(P)

,Value
name,VALID-ROBUST-OCSVM
frequency,100
max_subjects,29
max_test_subjects,10
user_ids,"[1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 2..."
num_sample_points_per_exp,21000
exp_begin_cutoff_idx,500
exp_end_cutoff_idx,-500
seconds_per_subject_train,210
seconds_per_subject_test,210


In [18]:
utils_eer_scorer

make_scorer(utils_eer, greater_is_better=False)

In [19]:
#include 47 later
# user_ids = [9]
df_exps_dict = load_data_frames(P.user_ids, P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, P.num_sample_points_per_exp)
raw_dfList_exp1, raw_dfList_exp2 = df_exps_dict['dfList_exp1'], df_exps_dict['dfList_exp2']

Loading exp1 data:
1) accel_count: 28388, gyro_count: 31997
2) accel_count: 26010, gyro_count: 28954
3) accel_count: 28227, gyro_count: 31814
4) accel_count: 24860, gyro_count: 26105
5) accel_count: 24270, gyro_count: 24347
6) accel_count: 25012, gyro_count: 25060
7) accel_count: 25301, gyro_count: 25382
8) accel_count: 21975, gyro_count: 21658
19) accel_count: 24110, gyro_count: 25050
21) accel_count: 24326, gyro_count: 23809
22) accel_count: 29123, gyro_count: 28724
26) accel_count: 23148, gyro_count: 24291
27) accel_count: 24299, gyro_count: 23589
28) accel_count: 23807, gyro_count: 24523
29) accel_count: 24030, gyro_count: 23457
35) accel_count: 24388, gyro_count: 23673
36) accel_count: 24228, gyro_count: 24208
37) accel_count: 31945, gyro_count: 31816
38) accel_count: 22135, gyro_count: 22327
39) accel_count: 23573, gyro_count: 23459
40) accel_count: 23057, gyro_count: 24296
41) accel_count: 24102, gyro_count: 23681
42) accel_count: 24074, gyro_count: 24328
43) accel_count: 22631,

In [20]:
num_sample_points_per_exp_user_47 = 18000
df_exps_dict_user_47 = load_data_frames([47], P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, num_sample_points_per_exp_user_47)
dfList_exp1_user_47, dfList_exp2_user_47 = df_exps_dict_user_47['dfList_exp1'], df_exps_dict_user_47['dfList_exp2']

raw_dfList_exp1_user_47 = dfList_exp1_user_47
raw_dfList_exp2_user_47 = dfList_exp2_user_47

Loading exp1 data:
47) accel_count: 22777, gyro_count: 22226
Loading exp2 data:
47) accel_count: 17718, gyro_count: 18353


# 1. Butterworth frequency Cut-off

## 1.1 Naive Approach
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    









EER_df_train_dict={}
EER_df_test_dict={}
for cut_off_freq in tqdm(list(range(48, 50))):
    P.cut_off_freq=cut_off_freq
    print(f"cut_off_freq: {P.cut_off_freq}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    
    dfList_dict={
                "dfList_exp1": ffted_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": ffted_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.cut_off_freq] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                                    extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                                    param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq: {P.cut_off_freq}\n")
        f.write(EER_df_train_dict[P.cut_off_freq].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

## 1.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)
P.smoothing = "Butterworth"

preprocessing_method="Realworld-per_unknown_window"

time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"

with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")

clf_type="standalone"


time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")



for cv_fold_idx in range(len(THREE_FOLD_CV)):
    
    train_set, test_set=THREE_FOLD_CV[cv_fold_idx]
    print(f"train_set: {train_set}")
    print(f"test_set: {test_set}")
    
    #--------------butter33----------------------
    P.cut_off_freq=33

    P.Butter_per_win_argdict={
        "filter_order": P.filter_order,
        "cut_off_freq": P.cut_off_freq,
        "sampling_freq": P.sampling_freq,
        "filtfilt": P.filtfilt,
        }   


    print(f"cut_off_freq: {P.cut_off_freq}")

    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)


    #--------------butter33----------------------

    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }
        
        

    # save_info_dict={"model_classifier_type": f"SCNN-{clf_type}", "exp_num": EXP_NUM_1_2, "cv_fold_idx": cv_fold_idx}
    # model_training_with_training_config_dict(dfList_dict=dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, overlap=OVERLAP, 
    #                                          training_config_dict=get_training_config_dict('Butter', cv_fold_idx), save_info_dict=save_info_dict)


    exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_1_2, 
                                                                        model_architectures=list(TRAINING_CONFIG_CV_DICT["Butter"][cv_fold_idx]['1000'].keys()), 
                                                                        exp_path_name=EXP_PATH_NAME,
                                                                        window_size_lst=WINDOW_SIZE_LST_NN,
                                                                        variant_suffix=f"Butter33-cv{cv_fold_idx}")
        

    key_column= ["cut_off_freq"]
    # -----CV_FOLD-------
    # for cv_fold_idx in range(len(THREE_FOLD_CV)):
    process_cv_fold_OCSVM_LOF_IF_NN(\
                                    cv_fold_idx=cv_fold_idx, 
                                    cv_sets=THREE_FOLD_CV, 
                                    dfList_dict=dfList_dict, 
                                    window_size_lst=WINDOW_SIZE_LST_NN, 
                                    exp_config=P, 
                                    exp_deep_archi_extractors_dict=exp_deep_archi_extractors_dict, 
                                    overlap=OVERLAP, 
                                    param_dist=param_dist, 
                                    train_file_name=train_file_name, 
                                    test_file_name=test_file_name, 
                                    preprocessing_params=P.cut_off_freq,
                                    key_column=key_column,
                                   )
    
    
    del exp_deep_archi_extractors_dict
    gc.collect()

reseting experiment params successful!
train_set: {0, 1, 2, 3, 4, 5, 6, 9, 10, 12, 14, 15, 16, 18, 19, 22, 23, 24, 25, 28}
test_set: {7, 8, 11, 13, 17, 20, 21, 26, 27, 29}
cut_off_freq: 33
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/1.2/SCNN_3_1_conv_1_dense_arg_dict_default/125/SCNN-standalone-1.2-SCNN_3_1_conv_1_dense_arg_dict_default-125-deep_feature_extractor-Butter33-cv0


2023-08-03 05:13:14.792048: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-08-03 05:13:14.792125: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2023-08-03 05:13:14.792563: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/1.2/SCNN_3_1_conv_1_dense_arg_dict_default/250/SCNN-standalone-1.2-SCNN_3_1_conv_1_dense_arg_dict_default-250-deep_feature_extractor-Butter33-cv0
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/1.2/SCNN_3_1_conv_1_dense_arg_dict_default/500/SCNN-standalone-1.2-SCNN_3_1_conv_1_dense_arg_dict_default-500-deep_feature_extractor-Butter33-cv0
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/1.2/SCNN_3_1_conv_1_dense_arg_dict_default/750/SCNN-standalone-1.2-SCNN_3_1_conv_1_dense_arg_dict_default-750-deep_feature_extractor-Butter33-cv0
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/1.2/SCNN_3_1_conv_1_dense_arg_dict_default/1000/SCNN-standalone-1.2-SCNN_3_1_conv_1_dense_arg_dict_default-1000-deep_feature_extractor-Butter33-cv0
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/1.2/SCNN_3_2_conv_1_dense_arg_dict_default/125/SCNN-standalone-1.2-SCNN_3_2_con

  0%|          | 0/5 [00:00<?, ?it/s]

window_size: 125
NN preprocessing
len(exp1_df_user_set_dict): 20
len(exp2_df_user_set_dict): 20
[-1. -1. -1. -1. -1. -1.]
[0.99999994 1.         1.0000002  0.99999994 1.         0.9999999 ]
[-1. -1. -1. -1. -1. -1.]
[0.9999999  0.99999994 0.99999994 1.         1.         1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.0000001  1.         0.9999998  0.99999994 1.         1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.        1.        1.        1.0000001 0.9999998 1.       ]
[-1. -1. -1. -1. -1. -1.]
[0.9999999  0.9999999  1.0000001  0.9999999  1.         0.99999994]
[-1. -1. -1. -1. -1. -1.]
[1.         0.99999994 0.99999994 1.         0.99999994 0.9999999 ]
[-1. -1. -1. -1. -1. -1.]
[1.         1.0000001  1.0000002  0.99999994 0.99999994 1.0000001 ]
[-1. -1. -1. -1. -1. -1.]
[1.0000001 0.9999998 0.9999998 1.0000001 1.        0.9999998]
[-1. -1. -1. -1. -1. -1.]
[1.         0.9999998  1.         0.9999998  0.99999994 1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.0000001  0.9999999  0.99999994 1.        

...


  0%|          | 0/3 [00:00<?, ?it/s]

run: 0: 100%|██████████| 20/20 [00:00<00:00, 2843.60it/s]

 33%|███▎      | 1/3 [15:39<31:18, 939.15s/it]

run: 1: 100%|██████████| 20/20 [00:00<00:00, 14637.25it/s]

 67%|██████▋   | 2/3 [26:26<12:47, 767.50s/it]

run: 2: 100%|██████████| 20/20 [00:00<00:00, 10558.35it/s]

100%|██████████| 3/3 [44:34<00:00, 891.42s/it]

  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 20/20 [00:00<00:00, 19868.80it/s]

 20%|██        | 1/5 [00:17<01:09, 17.35s/it]

run: 1: 100%|██████████| 20/20 [00:00<00:00, 12028.40it/s]

 40%|████      | 2/5 [00:39<01:00, 20.29s/it]

run: 2: 100%|██████████| 20/20 [00:00<00:00, 9039.45it/s]

 60%|██████    | 3/5 [01:01<00:41, 20.87s/it]

run: 3: 100%|██████████| 20/20 [00:00<00:00, 11964.92it/s]

 80%|████████  | 4/5 [01:28<00:23, 23.51s/it]

run: 4: 100%|██████████| 20/20 [00:00<00:00, 6252.22it/s]

 20%|██        | 1/5 [54:40<3:38:40, 3280.05s/it]

window_size: 250
NN preprocessing
len(exp1_df_user_set_dict): 20
len(exp2_df_user_set_dict): 20
[-1. -1. -1. -1. -1. -1.]
[0.99999994 1.         1.0000002  0.99999994 1.         0.9999999 ]
[-1. -1. -1. -1. -1. -1.]
[0.9999999  0.99999994 0.99999994 1.         1.         1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.0000001  1.         0.9999998  0.99999994 1.         1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.        1.        1.        1.0000001 0.9999998 1.       ]
[-1. -1. -1. -1. -1. -1.]
[0.9999999  0.9999999  1.0000001  0.9999999  1.         0.99999994]
[-1. -1. -1. -1. -1. -1.]
[1.         0.99999994 0.99999994 1.         0.99999994 0.9999999 ]
[-1. -1. -1. -1. -1. -1.]
[1.         1.0000001  1.0000002  0.99999994 0.99999994 1.0000001 ]
[-1. -1. -1. -1. -1. -1.]
[1.0000001 0.9999998 0.9999998 1.0000001 1.        0.9999998]
[-1. -1. -1. -1. -1. -1.]
[1.         0.9999998  1.         0.9999998  0.99999994 1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.0000001  0.9999999  0.99999994 1.        


  0%|          | 0/3 [00:00<?, ?it/s]

run: 0: 100%|██████████| 20/20 [00:00<00:00, 4089.01it/s]

 33%|███▎      | 1/3 [11:24<22:49, 684.64s/it]

run: 1: 100%|██████████| 20/20 [00:00<00:00, 12818.78it/s]

 67%|██████▋   | 2/3 [30:09<15:43, 943.64s/it]

run: 2: 100%|██████████| 20/20 [00:00<00:00, 386.84it/s]

100%|██████████| 3/3 [47:20<00:00, 946.86s/it]

  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 20/20 [00:00<00:00, 10751.87it/s]

 20%|██        | 1/5 [00:15<01:02, 15.74s/it]

run: 1: 100%|██████████| 20/20 [00:00<00:00, 12405.51it/s]

 40%|████      | 2/5 [00:30<00:45, 15.08s/it]

run: 2: 100%|██████████| 20/20 [00:00<00:00, 12080.37it/s]

 60%|██████    | 3/5 [00:44<00:29, 14.79s/it]

run: 3: 100%|██████████| 20/20 [00:00<00:00, 9874.76it/s]

 80%|████████  | 4/5 [00:59<00:14, 14.66s/it]

run: 4: 100%|██████████| 20/20 [00:00<00:00, 11059.47it/s]

 40%|████      | 2/5 [1:46:40<2:39:18, 3186.31s/it]

window_size: 500
NN preprocessing
len(exp1_df_user_set_dict): 20
len(exp2_df_user_set_dict): 20
[-1. -1. -1. -1. -1. -1.]
[0.99999994 1.         1.0000002  0.99999994 1.         0.9999999 ]
[-1. -1. -1. -1. -1. -1.]
[0.9999999  0.99999994 0.99999994 1.         1.         1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.0000001  1.         0.9999998  0.99999994 1.         1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.        1.        1.        1.0000001 0.9999998 1.       ]
[-1. -1. -1. -1. -1. -1.]
[0.9999999  0.9999999  1.0000001  0.9999999  1.         0.99999994]
[-1. -1. -1. -1. -1. -1.]
[1.         0.99999994 0.99999994 1.         0.99999994 0.9999999 ]
[-1. -1. -1. -1. -1. -1.]
[1.         1.0000001  1.0000002  0.99999994 0.99999994 1.0000001 ]
[-1. -1. -1. -1. -1. -1.]
[1.0000001 0.9999998 0.9999998 1.0000001 1.        0.9999998]
[-1. -1. -1. -1. -1. -1.]
[1.         0.9999998  1.         0.9999998  0.99999994 1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.0000001  0.9999999  0.99999994 1.        


  0%|          | 0/3 [00:00<?, ?it/s]

run: 0: 100%|██████████| 20/20 [00:00<00:00, 10415.46it/s]


In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"


preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    







P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_train_dict={}
EER_df_test_dict={}

for cut_off_freq in tqdm(list(range(48, 50))):
    P.cut_off_freq=cut_off_freq
    P.Butter_per_win_argdict["cut_off_freq"]=cut_off_freq
    print(f"cut_off_freq: {P.cut_off_freq}")

    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.cut_off_freq] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                                    extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                                    param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq: {P.cut_off_freq}\n")
        f.write(EER_df_train_dict[P.cut_off_freq].to_string())



mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 2. Butterworth frequency Cut-off + EMA span
## 2.1 Naive Approach
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_span_pairs: {index_pairs.shape}, choice_num: {choice_num}")
cut_off_span_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]








EER_df_train_dict={}
EER_df_test_dict={}

for key_pair in tqdm(cut_off_span_pairs):
    
    key_pair = tuple(key_pair)
    cut_off_freq, span = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.span=span
    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"span: {P.span}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1 = get_EMAed_dfList(ffted_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(ffted_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, span: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")
        

## 2.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)
P.smoothing = "Butter+EMA"

preprocessing_method="Realworld-per_unknown_window"

time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"

with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")

clf_type="standalone"


time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")



for cv_fold_idx in range(len(THREE_FOLD_CV)):
    
    train_set, test_set=THREE_FOLD_CV[cv_fold_idx]
    print(f"train_set: {train_set}")
    print(f"test_set: {test_set}")
    
    #--------------butter33-EMA20----------------------

    P.cut_off_freq=33
    P.span=20

    P.Butter_per_win_argdict={
        "filter_order": P.filter_order,
        "cut_off_freq": P.cut_off_freq,
        "sampling_freq": P.sampling_freq,
        "filtfilt": P.filtfilt,
        }

    P.EMA_per_win_span=P.span

    print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")


    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)

    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)

    #--------------butter33-EMA20----------------------

    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
        



    exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_2_2, 
                                                                        model_architectures=list(TRAINING_CONFIG_CV_DICT["Butter-EMA"][cv_fold_idx]['1000'].keys()), 
                                                                        exp_path_name=EXP_PATH_NAME,
                                                                        window_size_lst=WINDOW_SIZE_LST_NN,
                                                                        variant_suffix=f"Butter33-EMA20-cv{cv_fold_idx}")
        

    key_column= ["cut_off_freq", "EMA_span"]
    # -----CV_FOLD-------
    # for cv_fold_idx in range(len(THREE_FOLD_CV)):
    process_cv_fold_OCSVM_LOF_IF_NN(\
                                    cv_fold_idx=cv_fold_idx, 
                                    cv_sets=THREE_FOLD_CV, 
                                    dfList_dict=dfList_dict, 
                                    window_size_lst=WINDOW_SIZE_LST_NN, 
                                    exp_config=P, 
                                    exp_deep_archi_extractors_dict=exp_deep_archi_extractors_dict, 
                                    overlap=OVERLAP, 
                                    param_dist=param_dist, 
                                    train_file_name=train_file_name, 
                                    test_file_name=test_file_name, 
                                    preprocessing_params=(P.cut_off_freq, P.EMA_per_win_span),
                                    key_column=key_column,
                                   )
    
    
    del exp_deep_archi_extractors_dict
    gc.collect()

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_span_pairs: {index_pairs.shape}, choice_num: {choice_num}")
cut_off_span_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]






P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_train_dict={}
EER_df_test_dict={}

for key_pair in tqdm(cut_off_span_pairs):
    key_pair = tuple(key_pair)
    cut_off_freq, span = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.Butter_per_win_argdict["cut_off_freq"]=cut_off_freq
    
    
    P.span=span
    P.EMA_per_win_span=span

    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"span: {P.span}")

    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, span: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 3. EMA span
## 3.1 Naive Approach
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    




EER_df_train_dict={}
EER_df_test_dict={}
for span in tqdm(list(range(48, 50))):
    P.span=span
    print(f"EMA span: {P.span}")

    
    EMAed_dfList_exp1 = get_EMAed_dfList(raw_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(raw_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.span] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                    extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                    param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\EMA span: {P.span}\n")
        f.write(EER_df_train_dict[P.span].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

## 3.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    




EER_df_train_dict={}
EER_df_test_dict={}
for span in tqdm(list(range(48, 50))):
    P.span=span
    print(f"EMA span: {P.span}")

    P.EMA_per_win_span=P.span

    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.span] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                    extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                    param_dist=param_dist) 
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\EMA span: {P.span}\n")
        f.write(EER_df_train_dict[P.span].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 4. SMA winsize
## 4.1 Naive Approach
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
  






EER_df_train_dict={}
EER_df_test_dict={}
for winsize in tqdm(list(range(48, 50))):
    P.winsize=winsize
    print(f"SMA winsize: {P.winsize}")


    SMAed_dfList_exp1 = get_SMAed_dfList(raw_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(raw_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.winsize] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                       extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                       param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\SMA winsize: {P.winsize}\n")
        f.write(EER_df_train_dict[P.winsize].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

## 4.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    





EER_df_train_dict={}
EER_df_test_dict={}
for winsize in tqdm(list(range(48, 50))):
    P.winsize=winsize
    P.SMA_per_win_winsize=P.winsize

    print(f"SMA winsize: {P.winsize}")


    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[P.winsize] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                       extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                       param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\SMA winsize: {P.winsize}\n")
        f.write(EER_df_train_dict[P.winsize].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 5. Butterworth frequency Cut-off + SMA winsize
## 5.1 Naive Approach
### Optimizing and Testin

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_winsize_pairs: {index_pairs.shape}, choice_num: {choice_num}")
cut_off_winsize_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]








EER_df_train_dict={}
EER_df_test_dict={}

for key_pair in tqdm(cut_off_winsize_pairs):
    
    key_pair = tuple(key_pair)
    cut_off_freq, winsize = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.winsize=winsize
    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"winsize: {P.winsize}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1 = get_SMAed_dfList(ffted_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(ffted_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, winsize: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

## 5.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)
P.smoothing = "Butter+SMA"

preprocessing_method="Realworld-per_unknown_window"

time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"

with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")


clf_type="standalone"




for cv_fold_idx in range(len(THREE_FOLD_CV)):
    
    train_set, test_set=THREE_FOLD_CV[cv_fold_idx]
    print(f"train_set: {train_set}")
    print(f"test_set: {test_set}")
    
    # --------------butter33-SMA20----------------------
    P.winsize=20
    P.cut_off_freq=33

    P.Butter_per_win_argdict={
        "filter_order": P.filter_order,
        "cut_off_freq": P.cut_off_freq,
        "sampling_freq": P.sampling_freq,
        "filtfilt": P.filtfilt,
         }   
    P.SMA_per_win_winsize=P.winsize

    print(f"cut_off_freq: {P.cut_off_freq}, winsize: {P.winsize}")




    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)

    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)


    # --------------butter33-SMA20----------------------

    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
        



    exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_5_2, 
                                                                        model_architectures=list(TRAINING_CONFIG_CV_DICT["Butter-SMA"][cv_fold_idx]['1000'].keys()), 
                                                                        exp_path_name=EXP_PATH_NAME,
                                                                        window_size_lst=WINDOW_SIZE_LST_NN,
                                                                        variant_suffix=f"Butter33-SMA20-cv{cv_fold_idx}")
        

    key_column= ["cut_off_freq", "SMA_winsize"]
    # -----CV_FOLD-------
    # for cv_fold_idx in range(len(THREE_FOLD_CV)):
    process_cv_fold_OCSVM_LOF_IF_NN(\
                                    cv_fold_idx=cv_fold_idx, 
                                    cv_sets=THREE_FOLD_CV, 
                                    dfList_dict=dfList_dict, 
                                    window_size_lst=WINDOW_SIZE_LST_NN, 
                                    exp_config=P, 
                                    exp_deep_archi_extractors_dict=exp_deep_archi_extractors_dict, 
                                    overlap=OVERLAP, 
                                    param_dist=param_dist, 
                                    train_file_name=train_file_name, 
                                    test_file_name=test_file_name, 
                                    preprocessing_params=(P.cut_off_freq, P.SMA_per_win_winsize),
                                    key_column=key_column,
                                   )
    
    
    del exp_deep_archi_extractors_dict
    gc.collect()

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    
indices = list(range(1, 50))
mesh = np.array(np.meshgrid(indices, indices))
index_pairs = mesh.T.reshape(-1, 2)

print(f"total cut_off_winsize_pairs: {index_pairs.shape}, choice_num: {choice_num}")
cut_off_winsize_pairs = index_pairs[np.random.choice(index_pairs.shape[0], size=CHOICE_NUM_PAIRS, replace=False), :]







P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_train_dict={}
EER_df_test_dict={}

for key_pair in tqdm(cut_off_winsize_pairs):
    
    key_pair = tuple(key_pair)
    cut_off_freq, winsize = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    P.winsize=winsize
    P.SMA_per_win_winsize=P.winsize
    
    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"winsize: {P.winsize}")

    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_train_dict[key_pair] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                      extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                      param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\cut_off_freq, winsize: {key_pair}\n")
        f.write(EER_df_train_dict[key_pair].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# 6. The effect of Varying Overlap

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")



time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    




EER_df_train_dict={}
EER_df_test_dict={}
for overlap in tqdm(list(range(51, 53))):
    overlap*=0.01
    max_window_size=2000
    step_width = int(max_window_size * (1-overlap))
    max_num_windows=max(len(getIndices(sampleSize=max_window_size, step=step_width, numSamplePoints=P.num_sample_points_per_exp)), 100)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": raw_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": raw_dfList_exp2_user_47
    }
    

    EER_df_train_dict[overlap] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                     extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                     param_dist=param_dist)
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\overlap: {overlap}\n")
        f.write(EER_df_train_dict[overlap].to_string())



        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=EXTRACT_WACA_features_DICT, overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["overlap"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

# Summary

In [ ]:
df=return_and_save_final_result_df_as_json_NN(final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, window_size_lst=WINDOW_SIZE_LST_NN)
df

In [ ]:
relative_df=return_and_save_final_relative_result_df_as_json(df, base_case_index=0, final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, window_size_lst=WINDOW_SIZE_LST=window_size_lst)
relative_df

In [ ]:
df.style.hide(axis='index').to_latex()


In [ ]:
relative_df.style.hide(axis='index').to_latex()
